In [1]:
import pandas as pd
import numpy as np

# Read South Foehn datasets and stack

In [2]:
df_1981_1990 = pd.read_csv("/net/litho/atmosdyn2/chmony/data/SouthFoehnData/ALT_1981_1990.dat", delimiter="\t", header=1)
df_1981_1990["date"] = pd.to_datetime(df_1981_1990["Datumsangabe_Zeit"], format='%Y%m%d %H:%M')
df_1981_1990 = df_1981_1990[["date", "Foehn"]]

In [3]:
df_1983_2019 = pd.read_csv("/net/litho/atmosdyn2/chmony/data/SouthFoehnData/ALT_1983_2019.dat", delimiter="|", skiprows=1, header=0, names=range(1,10))
df_1983_2019["date"] = pd.to_datetime(df_1983_2019[2], format='%Y%m%d%H%M%S')
df_1983_2019["Foehn"] = df_1983_2019[4]
df_1983_2019 = df_1983_2019[["date", "Foehn"]]

/home/chmony/Apps/anaconda3/envs/develop_xgboost/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# Stack dataframes. Take the years 1981 and 1982 from df_1981_1990 and the remainder from df_1983_2019
df_1981_2019 = pd.concat([df_1981_1990.loc[df_1981_1990["date"] < np.datetime64("1983-01-01 00:00:00")], df_1983_2019], axis=0, ignore_index=True)

In [5]:
# Ensure continuoes and consistent values in the "date" column
df_timeframe = pd.Series(pd.date_range(start = "1981-01-01 00:00:00", end="2019-12-31 23:50:00", freq="10min"), name="date")
df_1981_2019 = pd.merge(df_timeframe, df_1981_2019, on="date", how="left")

# Preprocess data

In [6]:
# Set all values in Foehn which are larger than 2 to np.NaN
df_1981_2019["Foehn"] = df_1981_2019["Foehn"].mask(df_1981_2019["Foehn"] >2)

# Set all Foehn values (Foehn==2) to 1 (i.e. treat Mischluft as normal foehn)
df_1981_2019["Foehn"] = df_1981_2019["Foehn"].mask(df_1981_2019["Foehn"] == 2.0, 1.0)

In [7]:
# Define rolling window of length 6. If at least 4 dates show Foehn say there is foehn prevalent, otherwise not (refer to Gutermann et. al.(2013))
# Allow max 2 missing values (-> min_periods=4). Otherwise set entry to np.NaN
foehn_rolling_window=df_1981_2019["Foehn"].rolling(window=6, min_periods=4).sum().shift(-3)
foehn_new_representation = (foehn_rolling_window >= 4).astype(int)
foehn_new_representation.loc[foehn_rolling_window.isnull()] = np.NaN

df_rolling= df_1981_2019.copy()
df_rolling["Foehn"]= foehn_new_representation

In [8]:
# Keep only timestamps at full hour and where hour equals 0,6,12 or 18
date_mask = (df_rolling["date"].dt.minute==0) & \
            ((df_rolling["date"].dt.hour == 0) | 
            (df_rolling["date"].dt.hour == 6) |
            (df_rolling["date"].dt.hour == 12) |
            (df_rolling["date"].dt.hour == 18))
df_foehn =df_rolling.loc[date_mask]

In [9]:
df_foehn.to_csv("/net/litho/atmosdyn2/chmony/data/SouthFoehnData/SouthFoehnDataProcessed.csv", index=False)

# Merge with meteorological data

In [10]:
df_ERA = pd.read_csv("/net/litho/atmosdyn2/chmony/data/MeteorologicalData/ERAI_data.csv.gz")
df_ERA["date"] = pd.to_datetime(df_ERA["date"], format="%Y-%m-%dT%H:%M:00.000000Z")

In [11]:
df_merged = pd.merge(left=df_foehn, right = df_ERA, how="inner", on="date")

In [12]:
df_merged.to_csv("/net/litho/atmosdyn2/chmony/data/FoehnAndMeteorologicalData/ERAI_and_Foehn_data.csv.gz", index=False, compression="infer")